In [1]:
#loads data into table
import os
import configparser
# get all the connections
username_AVS = 'verosdq'
password_AVS = 'verosdqrac'
SQL_LOAD=r'sqlldr userid='+username_AVS+'/\\"'+password_AVS+'\\"@\\"(DESCRIPTION =(ADDRESS = (PROTOCOL = TCP)(HOST = DB.DEV.VV.VEROSAPPS.COM)(PORT = 1521))(CONNECT_DATA =(SERVICE_NAME = VVDEVDB_TAF)))\\" control=E:\SQLLDR\DV_CALIBER.CTL'
os.system(SQL_LOAD)

2

In [3]:
#Loads data into AVS processing
#Output is number of transactions loaded
import csv
import cx_Oracle
import os
def get_order_count():
    try:
        # create a connection to the Oracle Database
        dsn_tns = cx_Oracle.makedsn('DB.DEV.VV.VEROSAPPS.COM', '1521', service_name='vvdevdb_taf') #if needed, place an 'r' before any parameter in order to address any special character such as '\'.
        with cx_Oracle.connect(user=username_AVS, password=password_AVS, dsn=dsn_tns) as connection:
            # create a new cursor
            with connection.cursor() as cursor:
                # create a new variable to hold the value of the
                # OUT parameter
                order_count = cursor.var(int)
                # call the stored procedure
                cursor.callproc('TRY_DV_AVS',[order_count])
                return order_count.getvalue()
    except cx_Oracle.Error as error:
        print(error)


if __name__ == '__main__':
   orders = get_order_count()
   print(orders)  # 3

3


In [4]:
#Checking whether AVS is complete or not, 1 - complete, 0- in process
import csv
import cx_Oracle
import os
def TRY_DV_AVS_CHECK1():
    try:
        # create a connection to the Oracle Database
        dsn_tns = cx_Oracle.makedsn('DB.DEV.VV.VEROSAPPS.COM', '1521', service_name='vvdevdb_taf') #if needed, place an 'r' before any parameter in order to address any special character such as '\'.
        with cx_Oracle.connect(user=username_AVS, password=password_AVS, dsn=dsn_tns) as connection:
            # create a new cursor
            with connection.cursor() as cursor:
                # create a new variable to hold the value of the
                # OUT parameter
                order_count = cursor.var(int)
                # call the stored procedure
                cursor.callproc('TRY_DV_AVS_CHECK1',[order_count])
                return order_count.getvalue()
    except cx_Oracle.Error as error:
        print(error)


if __name__ == '__main__':
   orders = TRY_DV_AVS_CHECK1()
   print(orders)  # 3

1


In [5]:
#Matches with AVM info main table and gets seqno 
#Matches with disaster data and identifies if property in the disaster
#Matches the A15 to get the VeroValue
#Output- total # of transactions client sent (from step 2)
#If it does not match then those have to be done manually as the next step
import csv
import cx_Oracle
import os
def TRY_DV_AVS_PROCESS_STEP1():
    try:
        # create a connection to the Oracle Database
        dsn_tns = cx_Oracle.makedsn('DB.DEV.VV.VEROSAPPS.COM', '1521', service_name='vvdevdb_taf') #if needed, place an 'r' before any parameter in order to address any special character such as '\'.
        with cx_Oracle.connect(user=username_AVS, password=password_AVS, dsn=dsn_tns) as connection:
            # create a new cursor
            with connection.cursor() as cursor:
                # create a new variable to hold the value of the
                # OUT parameter
                order_count = cursor.var(int)
                # call the stored procedure
                cursor.callproc('TRY_DV_AVS_PROCESS_STEP1',[order_count])
                return order_count.getvalue()
    except cx_Oracle.Error as error:
        print(error)


if __name__ == '__main__':
   orders = TRY_DV_AVS_PROCESS_STEP1()
   print(orders)  # 3

3


In [6]:
#extract data from oracle for excel file processing
import pysftp
import sys
import os
import zipfile
import pandas as pd
import time
import pysftp
import sys
import os
import zipfile
import pandas as pd
import time
import configparser
import numpy as np
import cx_Oracle as orcCon
from cx_Oracle import DatabaseError
dsn_tns = orcCon.makedsn('DB.DEV.VV.VEROSAPPS.COM', '1521', service_name='vvdevdb_taf') #if needed, place an 'r' before any parameter in order to address any special character such as '\'.
conn = orcCon.connect(user=username_AVS, password=password_AVS, dsn=dsn_tns)
sql = 'SELECT * from  SBHUPATHIRAJU.DV_Final_step_table_stg'
Maindf = pd.read_sql(sql, conn)
df2=Maindf
columns = ['AVS_LATITUDE', 'AVS_LONGITUDE']
df2 = df2.drop(columns, axis=1)

In [7]:
#to get the map 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

import seaborn as sn
#Maindf.at[2,'DISASTER_INDICATOR']='INSIDE EVENT AND BUFFER'
waterfront=Maindf
waterfront_map = folium.Map(location=[37.6, -95.665],
                        zoom_start=4.35,
                        tiles='openstreetmap')
                        
for i in waterfront.index:
    lat = waterfront.AVS_LATITUDE[i]
    long = waterfront.AVS_LONGITUDE[i]
    if ( waterfront.DISASTER_INDICATOR[i]== 'INSIDE EVENT'):marker = folium.Marker([lat, long],icon=folium.Icon(color='red', icon='home', prefix='fa')).add_to(waterfront_map)
    elif (waterfront.DISASTER_INDICATOR[i]== 'INSIDE EVENT AND BUFFER'):marker = folium.Marker([lat, long],icon=folium.Icon(color='orange', icon='home', prefix='fa')).add_to(waterfront_map)
    else:marker = folium.Marker([lat, long],icon=folium.Icon(color='green', icon='home', prefix='fa')).add_to(waterfront_map)

waterfront_map.save(outfile= "test.html")
waterfront_map



In [8]:
df=df2
MODULE_PATH = "E:/Batch_process_clients/property_report/__init__.py"
MODULE_NAME = "property_report"
import importlib
import sys
spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)

In [9]:

from property_report import prop_report
from property_report import config
import os
import pandas as pd
# construct filename
xl_dir = config.xl_in_dir
xl_nm = xl_dir+config.xl_in_flnm
Data = df2
prop_rpt = prop_report.write_report(Data)
prop_rpt.create_key_col()

### create separate dataframe for Disaster_indicator = 'INSIDE EVENT'

In [10]:
prop_rpt.In_event_prop =prop_rpt.create_prop_dataset('INSIDE EVENT')

INSIDE EVENT Diaster type has 0 properties


### create separate dataframe for Disaster_indicator = 'INSIDE BUFFER'

In [11]:
prop_rpt.In_buffer_prop =prop_rpt.create_prop_dataset('INSIDE BUFFER')

INSIDE BUFFER Diaster type has 0 properties


### create separate dataframe for Disaster_indicator = 'OUTSIDE EVENT AND BUFFER'¶

In [12]:
prop_rpt.out_event_buffer_prop= prop_rpt.create_prop_dataset('OUTSIDE EVENT AND BUFFER')

OUTSIDE EVENT AND BUFFER Diaster type has 3 properties


### create separate Aggregated dataframe for Disaster_indicator = 'INSIDE EVENT'

In [13]:
prop_rpt.inEventAgg  = prop_rpt.prop_calculation(prop_rpt.In_event_prop)

self.df['DISASTER_INDICATOR'].str.replace(,


### create separate Aggregated dataframe for Disaster_indicator = 'INSIDE BUFFER'

In [14]:
prop_rpt.inBufferAgg  =prop_rpt.prop_calculation(prop_rpt.In_buffer_prop)

self.df['DISASTER_INDICATOR'].str.replace(,


### create separate Aggregated dataframe for Disaster_indicator = 'OUTSIDE EVENT AND BUFFER'

In [15]:
prop_rpt.out_evt_bfr  =prop_rpt.prop_calculation(prop_rpt.out_event_buffer_prop)

self.df['DISASTER_INDICATOR'].str.replace(,


### prepare property_report page

In [16]:
prop_rpt.get_pivot()

### remove existing .xlsx files from output folder configured in configure.py

In [17]:
prop_rpt.file_cleanup()

 ['property_disaster_report_20230130.xlsx'] files are removed


### Write report into excel with formatting

In [18]:
prop_rpt.write_excel()

Report: E:/Batch_process_clients/property_disaster_report_20230206.xlsx is exported


C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:2902: UserWarning: Must have at least one data row in in add_table()
  warn("Must have at least one data row in in add_table()")
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1437: UserWarning: Image file 'E:/Batch_process_clients/testmap.jpg' not found.
  warn("Image file '%s' not found." % filename)
